<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [79]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup

In [80]:
params = {k:v.stip() for k, v in (t.split('=') for t in open('config.txt'))} # Считываю данные подключения и файла (делаю сокрытие данных ввиде логинов/паролей и т.п.)

In [81]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [82]:
query_3_1 = f'''
select
    count(*) vacansies_count
from
    VACANCIES
'''

In [83]:
vacansies_count = pd.read_sql_query(query_3_1, connection)
display(vacansies_count)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\671705815.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacansies_count = pd.read_sql_query(query_3_1, connection)


,vacansies_count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [84]:
query_3_2 = f'''
select
    count(*) employer_count
from
    EMPLOYERS
'''

In [85]:
employer_count = pd.read_sql_query(query_3_2, connection)
display(employer_count)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\3860600552.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employer_count = pd.read_sql_query(query_3_2, connection)


,employer_count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [86]:
query_3_3 = f'''
select
    count(*) area_count
from
    AREAS
'''

In [87]:
area_count = pd.read_sql_query(query_3_3, connection)
display(area_count)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\56774077.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  area_count = pd.read_sql_query(query_3_3, connection)


,area_count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [88]:
query_3_4 = f'''
select
    count(*) industries_count
from
    INDUSTRIES
'''

In [89]:
industries_count = pd.read_sql_query(query_3_4, connection)
display(industries_count)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\1188689664.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industries_count = pd.read_sql_query(query_3_4, connection)


,industries_count
0,294


***

* По данным первичного анализа видно:
1. Количество вакансий всего (49197), а кол-во работодателей (23501) это говорит о нехватке рабочей силы из-за малого кол-ва квалифицированных специалистов, ведь от 1 работодателя может быть предложенных рабочих мест до нескольких десятков.
2. Кол-во регионов довольно большое 1362.
3. Кол-во профессий всего 294 это говорит что найти работников будет довольно сложно, ведь по данным интернета в мире порядка 40 тыс. профессий. И размещенные вакаснии могут не подходить для работодателей.

Для лучшего понимания нам нужно более детально проанализировать данные

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [90]:
query_4_1 = f'''
SELECT 
    a.name AS area, COUNT(v.name) AS cnt
FROM 
    AREAS a
LEFT JOIN 
    VACANCIES v ON a.id = v.area_id
GROUP BY 
    a.name
ORDER BY
    cnt DESC
LIMIT 5
'''

In [91]:
cnt = pd.read_sql_query(query_4_1, connection)
display(cnt)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\3764292790.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cnt = pd.read_sql_query(query_4_1, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [92]:
query_4_2 = f'''
SELECT 
    COUNT(*) salary
FROM 
    VACANCIES
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [93]:
salary = pd.read_sql_query(query_4_2, connection)
display(salary)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\1624558400.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary = pd.read_sql_query(query_4_2, connection)


,salary
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [94]:
query_4_3 = f'''
SELECT 
    ROUND(AVG(salary_from)) max_mean,
    ROUND(AVG(salary_to)) min_mean
FROM 
    VACANCIES
'''

In [95]:
salary_mean = pd.read_sql_query(query_4_3, connection)
display(salary_mean)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\1774889949.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_mean = pd.read_sql_query(query_4_3, connection)


,max_mean,min_mean
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [96]:
query_4_4 = f'''
SELECT
    schedule,
    employment,
    COUNT(*) schudule_employment
FROM 
    VACANCIES
GROUP BY
    schedule,
    employment
ORDER BY schudule_employment DESC
'''

In [97]:
schudule_employment = pd.read_sql_query(query_4_4, connection)
display(schudule_employment)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\3014985911.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schudule_employment = pd.read_sql_query(query_4_4, connection)


,schedule,employment,schudule_employment
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [98]:
query_4_5 = f'''
SELECT
    experience ex_year,
    count(*)
FROM 
    VACANCIES
GROUP BY
    experience
ORDER BY 2 ASC
'''

In [99]:
ex_year = pd.read_sql_query(query_4_5, connection)
display(ex_year)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\3165436566.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ex_year = pd.read_sql_query(query_4_5, connection)


,ex_year,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

* Выводы по детальному анализу вакансий:
1. Топ 5 кол-во вакансий находятся в городах милиониках это видно по выведеннуму запросу в задании 1.
2. Половина кандидатов имеет заполненые поля з/п ктр они хотят, это может говорить о том что они следят за рынком труда и знают среднюю з/п. А также видно по з/п вилке что колебание между нижней и верхней границей довольно большое 40 тыс.руб. это может говорить о разных вещах: колебание зарплат у разных работодателей, возможно соискатель желает максимульную зп если от него будут требовать командировки, а также переезд в другой город.
3. Большое кол-во соискателей ищут работу в связке "Полный день-Полная занятость"
4. А также видно что около 15% соискателей не имеют никакого опыта работы. Возможно это выпустившиеся студенты, те кто хочет сменить вид своей работы и он не указывает никакого опыта в данной профессии.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [100]:
query_5_1 = f'''
SELECT
    e.name employers_name,
    count(v.id) vac_count
FROM 
    EMPLOYERS e
LEFT JOIN
    VACANCIES v ON e.id = v.employer_id
GROUP BY
    e.name
ORDER BY
    2 DESC
LIMIT 5
'''

In [101]:
employers_name = pd.read_sql_query(query_5_1, connection)
display(employers_name)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\119924057.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_name = pd.read_sql_query(query_5_1, connection)


,employers_name,vac_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [102]:
query_5_2 = f'''
SELECT
    a.name employers_area_name,
    count(e.id) employers_count
FROM 
    AREAS a
LEFT JOIN
    EMPLOYERS e on e.area = a.id
LEFT JOIN
    VACANCIES v on a.id = v.area_id
GROUP BY
    1
HAVING 
    count(v.id) = 0
ORDER BY 
    employers_count DESC
'''

In [103]:
employers_area_name = pd.read_sql_query(query_5_2, connection)
display(employers_area_name)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\3063252318.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_area_name = pd.read_sql_query(query_5_2, connection)


,employers_area_name,employers_count
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Ростовская область,18
...,...,...
588,Струнино,0
589,Славгород (Алтайский край),0
590,Сорочинск,0
591,Ленинский (Тульская область),0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [104]:
query_5_3 = f'''
SELECT
    e.name AS employers_area,
    COUNT(DISTINCT v.area_id) AS area_count
FROM 
    EMPLOYERS e
JOIN
    VACANCIES v ON e.id = v.employer_id
GROUP BY
    e.name
ORDER BY
    area_count DESC
'''

In [105]:
employers_area = pd.read_sql_query(query_5_3, connection)
display(employers_area)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\57003849.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_area = pd.read_sql_query(query_5_3, connection)


,employers_area,area_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [106]:
query_5_4 = f'''
SELECT
    COUNT(DISTINCT e.id) AS employer_count
FROM 
    EMPLOYERS e
LEFT JOIN
    EMPLOYERS_INDUSTRIES e_i ON e.id = e_i.employer_id
WHERE
    e_i.industry_id IS NULL
'''

In [107]:
industry = pd.read_sql_query(query_5_4, connection)
display(industry)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\1759923270.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industry = pd.read_sql_query(query_5_4, connection)


,employer_count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [108]:
query_5_5 = f'''
SELECT
    e.id,
    e.name,
    COUNT(e_i.industry_id) AS industry_count
FROM 
    EMPLOYERS e
LEFT JOIN
    EMPLOYERS_INDUSTRIES e_i ON e.id = e_i.employer_id
GROUP BY
    e.id, e.name
HAVING
    COUNT(e_i.industry_id) = 4
ORDER BY
    name
'''

In [109]:
industry_count = pd.read_sql_query(query_5_5, connection)
display(industry_count)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\2898138212.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industry_count = pd.read_sql_query(query_5_5, connection)


,id,name,industry_count
0,800646,101 Интернет,4
1,763400,21vek.by,4
2,64174,2ГИС,4
3,29890,2К,4
4,1980996,4 пикселя +,4
...,...,...,...
1133,24365,ЮРИОН,4
1134,863335,ЮТИП Технологии,4
1135,1430484,ЯКласс,4
1136,1620049,ЯрНео,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [110]:
query_5_6 = f'''
SELECT
    COUNT(DISTINCT e.id) AS employer_count
FROM 
    EMPLOYERS e
JOIN
    EMPLOYERS_INDUSTRIES e_i ON e.id = e_i.employer_id
WHERE
    e_i.industry_id IN (
        SELECT id FROM INDUSTRIES WHERE name = 'Разработка программного обеспечения'
    )
'''

In [111]:
employer_count = pd.read_sql_query(query_5_6, connection)
display(employer_count)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\2272543430.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employer_count = pd.read_sql_query(query_5_6, connection)


,employer_count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [112]:
url = "https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8"
response = requests.get(url)
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
soup = BeautifulSoup(response.text, 'html.parser')
# Извлечение таблицы с городами-миллионниками
table = soup.find('table', {'class': 'standard sortable jquery-tablesorter'})

# Получение списка городов из таблицы
cities = []
HTML_data = soup.find_all("table")[0].find_all("tr")[1:] # Пропускаем заголовок таблицы
for row in HTML_data:
    city_cell = row.find_all('td')[1]  # Получаем ячейку с названием города
    city = city_cell.text.strip()  # Извлекаем текст из ячейки
    cities.append(city)

cities_set = tuple(cities)
display(cities_set)

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Красноярск',
 'Нижний Новгород',
 'Челябинск',
 'Уфа',
 'Самара',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [113]:
query_5_7 = f'''
SELECT 
    "Название города",
    cnt
FROM
    (SELECT 
        a.name AS "Название города",
    COUNT
        (v.id) AS cnt
    FROM 
        areas a 
    LEFT JOIN 
        vacancies v ON v.area_id = a.id
    LEFT JOIN 
        employers AS e ON e.id = v.employer_id
    WHERE 
        e.name = 'Яндекс'
        AND a.name IN {cities_set}
    GROUP BY 1 

    UNION ALL

    SELECT 
        'Total' AS "Название города",
        COUNT(v.id) AS cnt
    FROM 
        areas a 
    LEFT JOIN 
        vacancies v ON v.area_id = a.id
    LEFT JOIN 
        employers AS e ON e.id = v.employer_id
    WHERE 
        e.name = 'Яндекс'
        AND a.name IN {cities_set}
    GROUP BY 
        1) AS city
'''

In [114]:
city = pd.read_sql_query(query_5_7, connection)
display(city)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\3450362412.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  city = pd.read_sql_query(query_5_7, connection)


,Название города,cnt
0,Total,485
1,Волгоград,24
2,Воронеж,32
3,Екатеринбург,39
4,Казань,25
5,Краснодар,30
6,Красноярск,23
7,Москва,54
8,Нижний Новгород,36
9,Новосибирск,35


***

На основе представленных данных о работодателях можно сделать несколько выводов:

1. Лидеры по количеству вакансий:
   - Найбольшее количество вакансий (1933) предлагает Яндекс, что подтверждает его статус одного из ведущих работодателей в стране.
   - Следующими по количеству вакансий идут Ростелеком (491), Тинькофф (444), СБЕР (428) и Газпром нефть (331). Это указывает на высокую активность этих компаний в поиске сотрудников.

2. Географическая распределённость работодателей:
   - Наибольшее число работодателей сосредоточено в России (410), за ней следует Казахстан (207).
   - В пределах России более всего работодателей представлено в Московской области (75) и Краснодарском крае (19). Это может указывать на экономическую активность и возможности в этих регионах.

3. Концентрация вакансий по регионам:
   - Вакансии распределены неравномерно по городу. Москва, с 54 вакансиями, является лидером среди российских городов, за ней следует Санкт-Петербург с 42 вакансиями. Это подтверждает предполагаемую концентрацию рабочих мест в крупных городах.

4. Наличие работодателей в небольших населённых пунктах:
   - Существует множество работодателей, предлагающих единичные вакансии, даже в малых городах, таких как Струнино, Славгород и другие. Это может свидетельствовать о наличии маленьких фирм и индивидуальных предпринимателей, создающих рабочие места.

5. Конкуренция среди работодателей:
   - Крупные компании, такие как Яндекс, Ростелеком и Газпром нефть, продолжают сохранять лидерство, что может затруднять поиск работы для соискателей, если они особенно популярны среди работников.

6. Возможности для карьерного роста:
   - Наличие большого числа вакансий в крупных компаниях означает, что потенциальные соискатели могут рассчитывать на возможность карьерного роста.

Эти выводы могут помочь соискателям и исследователям рынка труда лучше понимать текущее состояние занятости и тенденции в различных регионах и отраслях.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [115]:
query_6_1 = f'''
SELECT
    count(v.id) as experience_count
FROM
    VACANCIES v
WHERE 
    LOWER(v.name) LIKE '%data%' 
    OR LOWER(v.name) LIKE '%данн%'
'''

In [116]:
experience_count = pd.read_sql_query(query_6_1, connection)
display(experience_count)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\369904572.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  experience_count = pd.read_sql_query(query_6_1, connection)


,experience_count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [117]:
query_6_2 = f'''
SELECT
    count(v.name) as ds_count
FROM
    VACANCIES v
WHERE
(
    (v.name ILIKE '%data scientist%') OR (v.name ILIKE '%data science%') OR (v.name ILIKE '%исследователь данных%')
    OR ((v.name ILIKE '%ML%') AND (v.name NOT ILIKE '%HTML%'))
    OR (v.name ILIKE '%machine learning%')
    OR (v.name ILIKE '%машинн%обучен%')
)
    AND (v.name ILIKE '%junior%'
    OR v.experience ILIKE '%Нет опыта%'
    OR v.employment ILIKE '%Стажировка%')
'''

In [118]:
ds_count = pd.read_sql_query(query_6_2, connection)
display(ds_count)

C:\Users\User\AppData\Local\Temp\ipykernel_30860\1008293209.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ds_count = pd.read_sql_query(query_6_2, connection)


,ds_count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [119]:
# текст запроса

In [120]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [121]:
# текст запроса

In [122]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [123]:
# текст запроса

In [124]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [125]:
# текст запроса

In [126]:
# результат запроса

***

In [127]:
# выводы по предметному анализу

# Общий вывод по проекту

In [128]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования